In [1]:
#모듈 호출

import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as spst
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")  # matplotlib style 설정

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#csv 파일 호출

df_orders=pd.read_csv("./data file/olist_orders.csv")
df_customers=pd.read_csv('./data file/olist_customers.csv')
df_order_items=pd.read_csv('./data file/olist_order_items.csv')
df_order_payments=pd.read_csv('./data file/olist_order_payments.csv')
df_products=pd.read_csv('./data file/olist_products.csv')
df_trans=pd.read_csv('./data file/product_category_name_translation.csv')

In [31]:
#전체 병합 파일 df 생성

df_co = pd.merge(df_customers, df_orders, on='customer_id', how='left')
df_co['total_orders'] = df_co.groupby('customer_unique_id')['order_id'].transform('count')
df_coi = pd.merge(df_co, df_order_items, on="order_id", how='left')
df_coi['total_order_items'] = df_coi.groupby('customer_unique_id')['customer_unique_id'].transform('count')
df_coip = pd.merge(df_coi, df_order_payments, on="order_id", how="left")
df_product=pd.merge(df_products,df_trans,on='product_category_name',how='outer')
mask = ~df_product['product_category_name_english'].isnull()
df_product.loc[mask, 'product_category_name'] = df_product.loc[mask, 'product_category_name_english']
df = pd.merge(df_coip, df_product, on="product_id", how="left")

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118434 entries, 0 to 118433
Data columns (total 25 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_id                    118434 non-null  object 
 1   customer_unique_id             118434 non-null  object 
 2   customer_zip_code_prefix       118434 non-null  int64  
 3   order_id                       118434 non-null  object 
 4   order_status                   118434 non-null  object 
 5   order_purchase_timestamp       118434 non-null  object 
 6   order_approved_at              118258 non-null  object 
 7   order_delivered_customer_date  115037 non-null  object 
 8   order_estimated_delivery_date  118434 non-null  object 
 9   total_orders                   118434 non-null  int64  
 10  product_id                     117604 non-null  object 
 11  seller_id                      117604 non-null  object 
 12  price                         

In [4]:
#한 번 구매한 고객 df_1 생성
df_1 = df[df['total_orders']==1]

#한 번 구매한 고객 제외 df_n 생성
df_n = df[df['total_orders']!=1]

In [5]:
df_n.head(1000)

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
5,879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,5741ea1f91b5fbab2bd2dc653a5b5099,delivered,2022-09-14 18:14:31,2022-09-14 18:25:11,2022-09-28 17:32:43,2022-10-04 00:00:00,2,...,1.0,debit_card,1.0,282.21,sports_leisure,4450.0,60.0,15.0,15.0,sports_leisure
8,5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,1ebeea841c590e86a14a0d7a48e7d062,delivered,2023-01-18 12:35:44,2023-01-18 12:56:32,2023-01-26 15:17:57,2023-02-20 00:00:00,2,...,1.0,debit_card,1.0,39.10,toys,350.0,28.0,10.0,12.0,toys
13,eabebad39a88bb6f5b52376faec28612,295c05e81917928d76245e842748184d,5704,852d2f4d37773bcbc21c8e09a05a4ea5,delivered,2023-03-07 15:57:14,2023-03-07 16:15:30,2023-03-15 00:46:30,2023-03-19 00:00:00,2,...,1.0,credit_card,3.0,36.39,telephony,300.0,17.0,4.0,12.0,telephony
35,2d5831cb2dff7cdefba62e950ae3dc7b,e9dd12dca17352644a959d9dea133935,42800,19c0c76962278b152cc44fc63ffd5e48,delivered,2023-04-18 09:42:53,2023-04-18 09:55:17,2023-05-17 20:19:49,2023-05-18 00:00:00,2,...,1.0,credit_card,1.0,67.05,furniture_decor,500.0,16.0,5.0,20.0,furniture_decor
36,b2bed119388167a954382cca36c4777f,e079b18794454de9d2be5c12b4392294,27525,77b062be7c5bd21712905feb8e1cfeed,delivered,2022-06-14 18:31:54,2022-06-15 18:43:04,2022-07-07 20:32:47,2022-07-07 00:00:00,2,...,1.0,credit_card,8.0,134.25,bed_bath_table,1050.0,16.0,10.0,16.0,bed_bath_table
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14758,5c495510e9f861b83aa5fb7c308e8d76,28989ef45087c96e5a4346e88216c2ba,69912,e660da13091dbe48784500c1c3e2fb75,delivered,2022-01-21 19:46:22,2022-01-21 20:01:49,2022-02-13 15:54:10,2022-03-07 00:00:00,2,...,1.0,credit_card,4.0,256.06,bed_bath_table,4700.0,45.0,18.0,35.0,bed_bath_table
14779,6b6d9a374c988f07d7da7033df7b0914,95828271d6bee6d5eedcd00e639d67a6,89281,96cf4805d2e98446035e05b0874255bb,delivered,2023-01-16 23:04:15,2023-01-16 23:13:21,2023-02-01 16:56:13,2023-02-09 00:00:00,2,...,1.0,credit_card,2.0,28.09,telephony,100.0,16.0,4.0,11.0,telephony
14787,5e3f8b9a2f9e4ea10ecf656c60412548,281e1e72207e518c29b249a1361a775d,9190,e7502198c4c3631116855ae6a8df1725,delivered,2022-05-08 11:59:25,2022-05-08 13:02:59,2022-05-12 11:24:49,2022-06-05 00:00:00,2,...,1.0,credit_card,2.0,275.96,air_conditioning,6100.0,42.0,42.0,35.0,air_conditioning
14792,f071974e65cb75a420c1530a8b0b27f4,4a6ccb379c9e5062ad3ce2324222fbf3,13181,e251afd3ced38d4de3a5ece05d615ef5,delivered,2023-01-25 14:59:08,2023-01-25 15:20:12,2023-02-04 19:35:39,2023-02-09 00:00:00,2,...,1.0,credit_card,1.0,21.43,electronics,200.0,22.0,11.0,16.0,electronics


In [6]:
df_n['order_purchase_timestamp'] = pd.to_datetime(df_n['order_purchase_timestamp'])

# 데이터를 시간 순서대로 정렬
df_n = df_n.sort_values(by=['customer_unique_id', 'order_purchase_timestamp'])

# 각 고객별로 주문 간 시간 간격을 계산하고 'duration_days' 열에 할당
df_n['duration_days'] = df_n.groupby('customer_unique_id')['order_purchase_timestamp'].diff().dt.days

# 첫 번째 주문의 'duration_days'를 NaN으로 처리
df_n.loc[df_n.groupby('customer_unique_id').cumcount() == 0, 'duration_days'] = pd.NaT


In [7]:
df_n

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,duration_days
23034,1afe8a9c67eec3516c09a8bdcc539090,00172711b30d52eea8b313a7f2cced02,45200,bb874c45df1a3c97842d52f31efee99a,delivered,2023-07-28 00:23:49,2023-07-28 00:35:19,2023-08-07 19:28:41,2023-08-22 00:00:00,2,...,credit_card,2.0,122.07,stationery,6250.0,25.0,50.0,25.0,stationery,NaT
42536,24b0e2bd287e47d54d193e7bbb51103f,00172711b30d52eea8b313a7f2cced02,45200,c306eca42d32507b970739b5b6a5a33a,canceled,2023-08-13 09:14:07,NaN,NaN,2023-09-05 00:00:00,2,...,voucher,1.0,122.07,NaN,NaN,NaN,NaN,NaN,NaN,16.0
23908,1b4a75b3478138e99902678254b260f4,004288347e5e88a27ded2bb23747066c,26220,a61d617fbe5bd006e40d3a0988fc844b,delivered,2022-07-27 14:13:03,2022-07-27 14:25:14,2022-08-04 16:26:54,2022-08-18 00:00:00,2,...,credit_card,2.0,251.09,cool_stuff,1625.0,44.0,24.0,32.0,cool_stuff,NaT
26360,f6efe5d5c7b85e12355f9d5c3db46da2,004288347e5e88a27ded2bb23747066c,26220,08204559bebd39e09ee52dcb56d8faa2,delivered,2023-01-14 07:36:54,2023-01-14 07:49:28,2023-01-19 22:14:14,2023-02-09 00:00:00,2,...,credit_card,2.0,103.28,luggage_accessories,700.0,19.0,48.0,35.0,luggage_accessories,170.0
86322,49cf243e0d353cd418ca77868e24a670,004b45ec5c64187465168251cd1c9c2f,57055,90ae229a4addcfead792e2564554f09c,shipped,2022-09-01 12:11:23,2022-09-05 04:30:20,NaN,2022-10-06 00:00:00,2,...,boleto,1.0,97.87,garden_tools,1850.0,37.0,22.0,40.0,garden_tools,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89499,1ae563fdfa500d150be6578066d83998,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,fd216c435fda2be27c92920a5e5dba82,delivered,2022-02-22 12:26:42,2022-02-22 13:05:25,2022-03-08 02:02:59,2022-03-23 00:00:00,3,...,credit_card,2.0,39.42,fashion_bags_accessories,150.0,16.0,5.0,11.0,fashion_bags_accessories,NaT
95172,d064be88116eb8b958727aec4cf56a59,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,5f79e2da24a52e76bc22897df03a7901,delivered,2022-08-23 13:15:29,2022-08-23 13:30:19,2022-09-25 21:34:47,2022-09-21 00:00:00,3,...,credit_card,2.0,57.60,bed_bath_table,1000.0,50.0,12.0,12.0,bed_bath_table,182.0
33385,bec0bf00ac5bee64ce8ef5283051a70c,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,6be1646fccbb69d34b22da3b1807d51d,delivered,2022-09-14 14:24:04,2022-09-14 14:39:31,2022-09-18 20:47:54,2022-10-02 00:00:00,3,...,credit_card,2.0,42.58,housewares,350.0,20.0,45.0,20.0,housewares,22.0
14458,0088395699ea0fcd459bfbef084997db,ffe254cc039740e17dd15a5305035928,37640,ebe76afae9242fc70b0b807b78f17adb,delivered,2022-04-02 16:33:30,2022-04-02 16:42:25,2022-04-20 09:33:20,2022-05-11 00:00:00,2,...,credit_card,2.0,28.42,bed_bath_table,200.0,30.0,4.0,40.0,bed_bath_table,NaT


In [8]:
#Nan 값과 하루에 많이 주문한 고객 제외
df_duration=df_n[df_n['duration_days']!=0]
df_duration = df_duration.dropna(subset=['duration_days'])

In [9]:
df_duration['duration_days'].median()

68.0

# Target & Sample

Target: 1번 구매 그룹+ 하루 n번 구매 그룹
- 이탈 고객 : 2023-08-11 이후 구매이력이 없는 집단
- 잠재 고객 : 2023-08-11 이후 구매이력이 있는 집단
(가장 최근 데이터  2023-10-18 기준으로 68일 이전)

Sample: n번 구매 그룹 - 하루 n번 구매 그룹

In [10]:
#총 고객 수: 96096명
df.groupby('customer_unique_id').size()

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    1
0000b849f77a49e4a4ce2b2a4ca5be3f    1
0000f46a3911fa3c0805444483337064    1
0000f6ccb0745a6a4b88665a16c9f078    1
0004aac84e0df4da2b147fca70cf8255    1
                                   ..
fffcf5a5ff07b0908bd4e2dbc735a684    2
fffea47cd6d3cc0a88bd621562a9d061    1
ffff371b4d645b6ecea244b27531430a    1
ffff5962728ec6157033ef9805bacc48    1
ffffd2657e2aad2907e67c3e9daecbeb    1
Length: 96096, dtype: int64

In [11]:
#n번 구매 그룹: 2997명
df_n.groupby('customer_unique_id').size()

customer_unique_id
00172711b30d52eea8b313a7f2cced02    2
004288347e5e88a27ded2bb23747066c    2
004b45ec5c64187465168251cd1c9c2f    2
0058f300f57d7b93c477a131a59b36c3    3
00a39521eb40f7012db50455bf083460    2
                                   ..
ff36be26206fffe1eb37afd54c70e18b    3
ff44401d0d8f5b9c54a47374eb48c1b8    4
ff8892f7c26aa0446da53d01b18df463    2
ff922bdd6bafcdf99cb90d7f39cea5b3    3
ffe254cc039740e17dd15a5305035928    2
Length: 2997, dtype: int64

In [12]:
#단일 방문 n번 구매한 적이 있는 그룹 'df_n_1day': 1549명 -> 877명
df_n_1day=df_n[df_n['duration_days']<=0]
df_n_1day.groupby('customer_unique_id').size()

customer_unique_id
0058f300f57d7b93c477a131a59b36c3    1
00cc12a6d8b578b8ebd21ea4e2ae8b27    1
011b4adcd54683b480c4d841250a987f    1
012452d40dafae4df401bced74cdb490    1
012a218df8995d3ec3bb221828360c86    1
                                   ..
fe52c2abb172426dd6416cc69e6b3689    1
fe613188d4d20ea3e3dc1290549c3ad1    1
fed2005ccab4fcf1a40ebdaff032a148    2
ff44401d0d8f5b9c54a47374eb48c1b8    3
ffe254cc039740e17dd15a5305035928    1
Length: 1549, dtype: int64

In [13]:
#재방문 구매한 적이 있는 그룹 'df_n_ndays': 2120명 #1448이어야하는데? -> 아니었음
df_duration.groupby('customer_unique_id').size()

customer_unique_id
00172711b30d52eea8b313a7f2cced02    1
004288347e5e88a27ded2bb23747066c    1
004b45ec5c64187465168251cd1c9c2f    1
0058f300f57d7b93c477a131a59b36c3    1
00a39521eb40f7012db50455bf083460    1
                                   ..
ff03923ad1eb9e32304deb7f9b2a45c9    1
ff22e30958c13ffe219db7d711e8f564    1
ff36be26206fffe1eb37afd54c70e18b    2
ff8892f7c26aa0446da53d01b18df463    1
ff922bdd6bafcdf99cb90d7f39cea5b3    2
Length: 2120, dtype: int64

In [14]:
#단일 방문 n번 구매한 후 재방문한 그룹!이 있는 거였다 : 672명
#이놈들 ㅜㅜㅜㅜㅜ 틀린줄 알았잖니

len(set(df_duration['customer_unique_id']).intersection(set(df_n_1day['customer_unique_id'])))

672

In [26]:
#단일 방문 'df_n_1day'에서 빌런들을 빼주자: 877명

villain=list(set(df_duration['customer_unique_id']).intersection(set(df_n_1day['customer_unique_id'])))
df_n_1day=df_n_1day[~df_n_1day['customer_unique_id'].isin(villain)]
df_n_1day.groupby('customer_unique_id').size()

customer_unique_id
00cc12a6d8b578b8ebd21ea4e2ae8b27    1
01a22e2079ea71e17313b88e5811e54a    2
01ea7dfdac01a4e8fbe2902b73510b20    1
027803eb28cc01fbdf5da72b109fabee    1
02b20b7c813efede140142ac610e36dc    1
                                   ..
fe28d443ab89dc67f5858bc2beda299a    1
fe613188d4d20ea3e3dc1290549c3ad1    1
fed2005ccab4fcf1a40ebdaff032a148    2
ff44401d0d8f5b9c54a47374eb48c1b8    3
ffe254cc039740e17dd15a5305035928    1
Length: 877, dtype: int64

In [27]:
#단일 방문 1번 구매 그룹: 93099명
df_1.groupby('customer_unique_id').size()

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    1
0000b849f77a49e4a4ce2b2a4ca5be3f    1
0000f46a3911fa3c0805444483337064    1
0000f6ccb0745a6a4b88665a16c9f078    1
0004aac84e0df4da2b147fca70cf8255    1
                                   ..
fffcf5a5ff07b0908bd4e2dbc735a684    2
fffea47cd6d3cc0a88bd621562a9d061    1
ffff371b4d645b6ecea244b27531430a    1
ffff5962728ec6157033ef9805bacc48    1
ffffd2657e2aad2907e67c3e9daecbeb    1
Length: 93099, dtype: int64

In [30]:
# df_n_1day의 customer_unique_id가 df_n에 존재하지 않는 행을 선택
villain=list(set(df_duration['customer_unique_id']).intersection(set(df_n_1day['customer_unique_id'])))
df_n_ndays = df_n[mask]
df_n_ndays

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,duration_days
23034,1afe8a9c67eec3516c09a8bdcc539090,00172711b30d52eea8b313a7f2cced02,45200,bb874c45df1a3c97842d52f31efee99a,delivered,2023-07-28 00:23:49,2023-07-28 00:35:19,2023-08-07 19:28:41,2023-08-22 00:00:00,2,...,credit_card,2.0,122.07,stationery,6250.0,25.0,50.0,25.0,stationery,NaT
42536,24b0e2bd287e47d54d193e7bbb51103f,00172711b30d52eea8b313a7f2cced02,45200,c306eca42d32507b970739b5b6a5a33a,canceled,2023-08-13 09:14:07,NaN,NaN,2023-09-05 00:00:00,2,...,voucher,1.0,122.07,NaN,NaN,NaN,NaN,NaN,NaN,16.0
23908,1b4a75b3478138e99902678254b260f4,004288347e5e88a27ded2bb23747066c,26220,a61d617fbe5bd006e40d3a0988fc844b,delivered,2022-07-27 14:13:03,2022-07-27 14:25:14,2022-08-04 16:26:54,2022-08-18 00:00:00,2,...,credit_card,2.0,251.09,cool_stuff,1625.0,44.0,24.0,32.0,cool_stuff,NaT
26360,f6efe5d5c7b85e12355f9d5c3db46da2,004288347e5e88a27ded2bb23747066c,26220,08204559bebd39e09ee52dcb56d8faa2,delivered,2023-01-14 07:36:54,2023-01-14 07:49:28,2023-01-19 22:14:14,2023-02-09 00:00:00,2,...,credit_card,2.0,103.28,luggage_accessories,700.0,19.0,48.0,35.0,luggage_accessories,170.0
86322,49cf243e0d353cd418ca77868e24a670,004b45ec5c64187465168251cd1c9c2f,57055,90ae229a4addcfead792e2564554f09c,shipped,2022-09-01 12:11:23,2022-09-05 04:30:20,NaN,2022-10-06 00:00:00,2,...,boleto,1.0,97.87,garden_tools,1850.0,37.0,22.0,40.0,garden_tools,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110777,62a2c35611e8757e40fbbcecb3c9b2bd,ff8892f7c26aa0446da53d01b18df463,45972,cb6e8d89049744886ad04520d5e0b2f6,delivered,2022-05-24 16:09:14,2022-05-24 16:22:42,2022-06-09 04:55:26,2022-06-26 00:00:00,2,...,credit_card,10.0,263.40,watches_gifts,400.0,16.0,11.0,12.0,watches_gifts,NaT
35359,e53e1eedc9ebf9fa7a3318c60ece1d25,ff8892f7c26aa0446da53d01b18df463,45972,1c308eca3f339414a92e518e2a2e5ee9,delivered,2022-11-26 23:25:43,2022-11-26 23:34:29,2023-01-23 14:37:27,2022-12-29 00:00:00,2,...,credit_card,6.0,66.74,bed_bath_table,350.0,25.0,9.0,19.0,bed_bath_table,186.0
89499,1ae563fdfa500d150be6578066d83998,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,fd216c435fda2be27c92920a5e5dba82,delivered,2022-02-22 12:26:42,2022-02-22 13:05:25,2022-03-08 02:02:59,2022-03-23 00:00:00,3,...,credit_card,2.0,39.42,fashion_bags_accessories,150.0,16.0,5.0,11.0,fashion_bags_accessories,NaT
95172,d064be88116eb8b958727aec4cf56a59,ff922bdd6bafcdf99cb90d7f39cea5b3,17340,5f79e2da24a52e76bc22897df03a7901,delivered,2022-08-23 13:15:29,2022-08-23 13:30:19,2022-09-25 21:34:47,2022-09-21 00:00:00,3,...,credit_card,2.0,57.60,bed_bath_table,1000.0,50.0,12.0,12.0,bed_bath_table,182.0


# 이제 Target과 Sample df를 만들어보자

In [40]:
#Target: 'df_1'+'df_n_1day' 
#93976명

#concat 위해 duration table drop
df_n_1day.drop('duration_days',axis=1,inplace=True)

df_Target=pd.concat([df_1, df_n_1day])
df_Target

,customer_id,customer_unique_id,customer_zip_code_prefix,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,total_orders,...,payment_sequential,payment_type,payment_installments,payment_value,product_category_name,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,00e7ee1b050b8499577073aeb2a297a1,delivered,2022-05-16 15:05:35,2022-05-16 15:22:12,2022-05-25 10:35:35,2022-06-05 00:00:00,1,...,1.0,credit_card,2.0,146.87,office_furniture,8683.0,54.0,64.0,31.0,office_furniture
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,29150127e6685892b6eab3eec79f59c7,delivered,2023-01-12 20:48:24,2023-01-12 20:58:32,2023-01-29 12:41:19,2023-02-06 00:00:00,1,...,1.0,credit_card,8.0,335.48,housewares,10150.0,89.0,15.0,40.0,housewares
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2023-05-19 16:07:45,2023-05-20 16:19:10,2023-06-14 17:58:51,2023-06-13 00:00:00,1,...,1.0,credit_card,7.0,157.73,office_furniture,8267.0,52.0,52.0,17.0,office_furniture
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,951670f92359f4fe4a63112aa7306eba,delivered,2023-03-13 16:06:38,2023-03-13 17:29:19,2023-03-28 16:04:25,2023-04-10 00:00:00,1,...,1.0,credit_card,1.0,173.30,office_furniture,12160.0,56.0,51.0,28.0,office_furniture
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2023-07-29 09:51:30,2023-07-29 10:10:09,2023-08-09 20:55:48,2023-08-15 00:00:00,1,...,1.0,credit_card,8.0,252.25,home_confort,5200.0,45.0,15.0,35.0,home_confort
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15437,70a9b3a2a2f6c0e8cc4852bc3a591e2e,fed2005ccab4fcf1a40ebdaff032a148,13870,02b0bf9164ac7753f9880e46dba17f6c,delivered,2022-06-17 17:04:31,2022-06-17 17:15:21,2022-06-30 12:22:11,2022-07-07 00:00:00,2,...,1.0,credit_card,1.0,63.42,computers_accessories,75.0,17.0,6.0,58.0,computers_accessories
18460,5d0c6c3c37720178553332f2509b2568,ff44401d0d8f5b9c54a47374eb48c1b8,13070,c087855fe6f303205c09558f7f8103c8,delivered,2022-05-19 21:20:54,2022-05-19 21:35:16,2022-06-01 18:05:18,2022-06-12 00:00:00,2,...,2.0,voucher,1.0,28.47,fashion_bags_accessories,200.0,16.0,9.0,11.0,fashion_bags_accessories
105234,09c682ef897935e677a0996cfe2008f3,ff44401d0d8f5b9c54a47374eb48c1b8,13070,e28a4c66f1484567d1e42b3fab64d923,delivered,2022-05-19 21:20:54,2022-05-19 21:42:44,2022-05-31 17:06:38,2022-06-23 00:00:00,2,...,1.0,credit_card,1.0,0.72,fashion_bags_accessories,150.0,16.0,9.0,11.0,fashion_bags_accessories
105235,09c682ef897935e677a0996cfe2008f3,ff44401d0d8f5b9c54a47374eb48c1b8,13070,e28a4c66f1484567d1e42b3fab64d923,delivered,2022-05-19 21:20:54,2022-05-19 21:42:44,2022-05-31 17:06:38,2022-06-23 00:00:00,2,...,2.0,voucher,1.0,38.28,fashion_bags_accessories,150.0,16.0,9.0,11.0,fashion_bags_accessories


In [41]:
df_Target.groupby('customer_unique_id').size()

customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    1
0000b849f77a49e4a4ce2b2a4ca5be3f    1
0000f46a3911fa3c0805444483337064    1
0000f6ccb0745a6a4b88665a16c9f078    1
0004aac84e0df4da2b147fca70cf8255    1
                                   ..
fffcf5a5ff07b0908bd4e2dbc735a684    2
fffea47cd6d3cc0a88bd621562a9d061    1
ffff371b4d645b6ecea244b27531430a    1
ffff5962728ec6157033ef9805bacc48    1
ffffd2657e2aad2907e67c3e9daecbeb    1
Length: 93976, dtype: int64

In [42]:
#Sample: 'df_n_ndays'
#2120명

df_Sample=df_n_ndays

NameError: name 'df_n_ndays' is not defined

# Target에서 이탈/잠재 고객을 나눠보자

### 잠재고객부터! df_poten

In [ ]:
# 잠재고객 'df_poten' 생성: 2023-08-11 이후 구매이력이 있는 집단

from datetime import datetime
standard=datetime(2023, 8, 11, 0, 0, 0)

df_Target['order_purchase_timestamp'] = pd.to_datetime(df_Target['order_purchase_timestamp'])

#기준시보다 크거나 같으면 poten
df_poten=df_Target[df_Target['order_purchase_timestamp'] >= standard]

In [ ]:
df_poten.info()

In [ ]:
#잠재고객 몇 명? : 3321명~~
#고객 머릿수로 뭔가 하고싶다면 꼭 groupby('customer_unique_id')맥일 것

df_poten.groupby('customer_unique_id').size()

### 이탈고객! 은 df_churn

In [ ]:
# 이탈고객 'df_churn' 생성: 2023-08-11 이후 구매이력이 없는 집단
#기준시보다 작으면? 으로 구하면 안됨.. 그 뒤로 또 주문했을 수도 있으니까 그냥 poten의 여집합으로 접근

poten=list(set(df_poten['customer_unique_id']))
df_churn=df_Target[~df_Target['customer_unique_id'].isin(poten)]                 

In [ ]:
#90655명~~

df_churn.groupby('customer_unique_id').size()

# 주문액 비교

가설 설정
귀무 가설: Target 그룹과 Sample 그룹은 주문 1회 당 금액 평균 차이가 없다 <br>
대립 가설: Target 그룹은 Sample 그룹은 주문 1회 당 금액 평균 차이가 있다.

?? 이상치를 정리해야할까?

In [ ]:
payment_value_Target=df_Target['payment_value'].dropna()
payment_value_Sample=df_Sample['payment_value'].dropna()

#T검정
t_stat, p_value = spst.ttest_ind(payment_value_Target, payment_value_Sample)

# 결과 출력
print(f"T-Statistic: {t_stat}, P-Value: {p_value}")

In [ ]:
#평균차 시각화

pvT=payment_value_Target
pvC=df_churn['payment_value'].dropna()
pvP=df_poten['payment_value'].dropna()
pvS=payment_value_Sample

# 그래프 크기 설정
plt.figure(figsize=(12, 8))

# 서브플롯 생성
plt.subplot(2, 2, 1)
sns.boxplot(data=pvT, notch=True)
plt.title('Boxplot of Payment Value (Target)')
plt.ylim(0, 8000)

plt.subplot(2, 2, 4)
sns.boxplot(data=pvC, notch=True)
plt.title('Boxplot of Payment Value (Churn)')
plt.ylim(0, 8000)

plt.subplot(2, 2, 3)
sns.boxplot(data=pvP, notch=True)
plt.title('Boxplot of Payment Value (Poten)')
plt.ylim(0, 8000)

plt.subplot(2, 2, 2)
sns.boxplot(data=pvS, notch=True)
plt.title('Boxplot of Payment Value (Sample)')

plt.ylim(0, 8000)

# 그래프 출력
plt.tight_layout()  # 서브플롯 간 간격 조정
plt.show()

In [21]:
print(f"Target 그룹 주문액 평균은 {payment_value_Target.mean()}이고,\nSample 그룹 주문액 평균은 {payment_value_Sample.mean()}이다.\n\n이탈 고객 주문액 평균은 {df_churn['payment_value'].dropna().mean()}이고, \n잠재 고객 주문액 평균은 {df_poten['payment_value'].dropna().mean()}이다.")


NameError: name 'payment_value_Target' is not defined